##### Overview:

Welcome to Day 14. 

Machine Learning (ML) may seem advanced, but you've already learned enough Python basics (Pandas, Numpy, and data visualization) to build your first ML models! 

In fact, you have learned enough to build your first GenAI RAG chatbot as well. 

You will learn:

- Build a RAG based question answer chatbot to chat with your own PDF.


#### 1: Install Dependencies


In [ ]:
#!pip3 install -U langchain langchain-openai openai


#### 2. Import Libraries

In [9]:
import os
import openai
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display

#### 3. Set Your OpenAI API Key

In [10]:
load_dotenv()
openai.api_key  = os.getenv("OPENAI_API_KEY")

#### 4. Load and Split PDF Text

In [ ]:
loader = PyPDFLoader("../data/The_alchemist_by_Paulo_Coelho.pdf")
docs = loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

#### 5. Embed documents and create a FAISS vector store

In [11]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

#### 6. Build the RetrievalQA Chain

In [12]:
retriever = vectorstore.as_retriever()

#### 7. Create a QA chain using ChatOpenAI

In [13]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Optional: Custom prompt template (optional for more control)
prompt_template = PromptTemplate.from_template(
    "Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template},
    input_key="question"
)

#### 8. Ask Question About Your PDF

In [16]:
question = "Who is main chacter in this book?"
response = qa_chain.invoke({"question": question}) 


#### 9. Show the answer

In [17]:
display(Markdown(response["result"]))

The main character in "The Alchemist" by Paulo Coelho is a young shepherd named Santiago. He embarks on a journey to discover his Personal Legend, which is his life's purpose or destiny.

# HAPPY LEARNING